In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt

# rolling minmax scaling 함수 (window=24)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

# binning 및 one-hot 인코딩 함수 (결과를 정수 0,1로)
def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        # pd.cut에 bins로 지정된 구간을 사용 (여기서는 자동으로 최소~최대 값)
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    # 개별 열 단위로 float32로 변환
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

####################################
# Dataset 및 모델 정의 (나머지 구성은 동일)
####################################
class TimeSeriesDataset(Dataset):
    def __init__(self, input_data, target_data, lookback=24):
        self.input_data = input_data.values
        self.target_data = target_data.values
        self.lookback = lookback

    def __len__(self):
        return len(self.input_data) - self.lookback

    def __getitem__(self, idx):
        x = self.input_data[idx: idx + self.lookback, :]
        y = self.target_data[idx + self.lookback, 0]
        y_prev = self.target_data[idx + self.lookback - 1, 0]
        y_target = 1 if y > y_prev else 0
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_target, dtype=torch.long)

class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])

####################################
# 데이터 로드 및 Variant 1 실행: OHLC 원본 데이터로 실험
####################################
# CSV 파일 로드 – OHLC 4개 데이터만 사용
data = pd.read_csv("BTC_upbit_KRW_min60.csv", index_col=0)
data = data[['open', 'high', 'low', 'close']]
data.index = pd.to_datetime(data.index)

# 각 OHLC 열에 대해 rolling minmax scaling (window=24)
ohlc_features = ['open', 'high', 'low', 'close']
for feature in ohlc_features:
    data[feature] = rolling_minmax_scale(data[feature], window=24)

# One-hot 인코딩: 각 OHLC 열은 100구간으로 나눔
data = bin_and_encode(data, ohlc_features, bins=100, drop_original=True)

# 타깃은 원본 close 값 사용 (실험 목적)
data['close_target'] = data['close']
data = data.dropna()

# 최종 입력은 _Bin_ 접미사 열들만 사용
final_input_columns = [col for col in data.columns if '_Bin_' in col]
final_target_column = ['close_target']

data_input = data[final_input_columns]
data_target = data[final_target_column]

# # (아래는 모델 학습/평가 루프 코드 – 생략 가능)
# print("Variant 1 - Raw OHLC with rolling minmax scaling and one-hot encoding")
# print("Final input shape:", data_input.shape)

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(data_loader), correct / total

def train_model(model, train_loader, val_loader, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def train_and_evaluate(data, num_experiments=16, lookback=24, num_epochs=10):
    # 최종 입력: 모든 _Bin_ 컬럼 + datetime one-hot + positional encoding + order 인코딩 컬럼
    final_input_columns = [col for col in data.columns if '_Bin_' in col]
    # datetime_onehot_features = [col for col in data.columns if col.startswith('Hour_') or 
    #                               col.startswith('Day_') or col.startswith('Week_') or 
    #                               col.startswith('Month_')]
    # datetime_positional_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos']
    # order_encoding_features = [col for col in data.columns if '_order_' in col]
    # final_input_columns.extend(datetime_onehot_features)
    # final_input_columns.extend(datetime_positional_features)
    # final_input_columns.extend(order_encoding_features)
    target_cols = ['close_target']
    
    data_input = data[final_input_columns]
    data_target = data[target_cols]
    
    data_input = data_input.apply(pd.to_numeric).astype(np.float32)
    data_target = data_target.apply(pd.to_numeric).astype(np.float32)
    
    step_size = 2500
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    val_acc_list = []
    test_acc_list = []
    
    for exp in range(num_experiments):
        train_start = exp * step_size
        train_end = train_start + step_size * 8
        val_end = train_end + step_size
        test_end = val_end + step_size
        if test_end > len(data_input):
            break
        print(f"Experiment {exp}: 데이터 구간 [{train_start}:{test_end}]")
        print(data)
        data.to_csv('BTC60_one_hot.csv')
        
        train_input = data_input.iloc[train_start:train_end]
        train_target = data_target.iloc[train_start:train_end]
        val_input = data_input.iloc[train_end:val_end]
        val_target = data_target.iloc[train_end:val_end]
        test_input = data_input.iloc[val_end:test_end]
        test_target = data_target.iloc[val_end:test_end]
        
        train_dataset = TimeSeriesDataset(train_input, train_target, lookback=lookback)
        val_dataset = TimeSeriesDataset(val_input, val_target, lookback=lookback)
        test_dataset = TimeSeriesDataset(test_input, test_target, lookback=lookback)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        lr = 1e-4
        input_dim = data_input.shape[1]
        model = EncoderOnlyTransformerCustom(input_dim=input_dim, embedding_dim=512, num_layers=6, nhead=8,
                                               ffn_dim=2048, num_classes=2, max_seq_len=lookback).to(device)
        model_path = f"model_experiment_{exp}.pth"
        if exp > 0:
            try:
                model.load_state_dict(torch.load(f"model_experiment_{exp - 1}.pth"))
                print(f"Loaded model from experiment {exp - 1} for fine-tuning.")
            except FileNotFoundError:
                print(f"Model file for experiment {exp - 1} not found. Starting fresh training.")
        
        print(f"Experiment {exp}: Training with lr={lr} (Fine-Tuning)")
        model = train_model(model, train_loader, val_loader, num_epochs, lr, device)
        torch.save(model.state_dict(), model_path)
        print(f"Saved model for experiment {exp}.")
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        val_acc_list.append(val_acc)
        print(f"Experiment {exp}: Final Validation Accuracy: {val_acc:.4f}")
        
        test_loss, test_acc = evaluate_model(model, test_loader, device)
        test_acc_list.append(test_acc)
        print(f"Experiment {exp}: Test Accuracy: {test_acc:.4f}")
    
    if val_acc_list:
        avg_val_acc = sum(val_acc_list) / len(val_acc_list)
        avg_test_acc = sum(test_acc_list) / len(test_acc_list)
        print(f"\nFinal Average Validation Accuracy: {avg_val_acc:.4f}")
        print(f"Final Average Test Accuracy: {avg_test_acc:.4f}")
    else:
        print("실험이 한 번도 실행되지 않았습니다.")

# 참조용 리스트 (최종 구성에는 _Bin_ 컬럼들이 포함됨)
# features_pct = [f + '_pct' for f in features_to_bin]
# features_minmax = [f + '_minmax' for f in features_to_bin]

train_and_evaluate(data)


C:\Users\User\AppData\Local\Temp\ipykernel_12720\3724248470.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['close_target'] = data['close']


Experiment 0: 데이터 구간 [0:25000]
                         open      high       low     close  open_Bin_0  \
2017-09-25 12:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 13:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 17:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 19:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 20:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
...                       ...       ...       ...       ...         ...   
2024-12-18 12:00:00  0.000000  0.000000  0.000000  0.000000         1.0   
2024-12-18 13:00:00  0.000000  0.000000  0.000000  0.000000         1.0   
2024-12-18 14:00:00  0.000000  0.000000  0.199575  0.000000         1.0   
2024-12-18 15:00:00  0.000000  0.000000  0.226983  0.036865         1.0   
2024-12-18 16:00:00  0.053565  0.041933  0.318664  0.109052         0.0   

                     open_Bin_1  open_Bin_2  open_Bin_3  open_Bin_4 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Experiment 0: Training with lr=0.0001 (Fine-Tuning)


c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


...

Loaded model from experiment 12 for fine-tuning.
Experiment 13: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6902, Train Acc: 0.5415 | Val Loss: 0.6904, Val Acc: 0.5368
Epoch 2/10 | Train Loss: 0.6903, Train Acc: 0.5414 | Val Loss: 0.6905, Val Acc: 0.5368
Epoch 3/10 | Train Loss: 0.6904, Train Acc: 0.5412 | Val Loss: 0.6908, Val Acc: 0.5368
Epoch 4/10 | Train Loss: 0.6903, Train Acc: 0.5395 | Val Loss: 0.6906, Val Acc: 0.5368
Epoch 5/10 | Train Loss: 0.6901, Train Acc: 0.5416 | Val Loss: 0.6917, Val Acc: 0.5368
Epoch 6/10 | Train Loss: 0.6903, Train Acc: 0.5420 | Val Loss: 0.6903, Val Acc: 0.5368
Epoch 7/10 | Train Loss: 0.6899, Train Acc: 0.5417 | Val Loss: 0.6903, Val Acc: 0.5368
Epoch 8/10 | Train Loss: 0.6897, Train Acc: 0.5416 | Val Loss: 0.6906, Val Acc: 0.5368
Epoch 9/10 | Train Loss: 0.6897, Train Acc: 0.5416 | Val Loss: 0.6905, Val Acc: 0.5368
Epoch 10/10 | Train Loss: 0.6900, Train Acc: 0.5418 | Val Loss: 0.6904, Val Acc: 0.5368
Saved model for experiment 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 13 for fine-tuning.
Experiment 14: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6906, Train Acc: 0.5399 | Val Loss: 0.6889, Val Acc: 0.5473
Epoch 2/10 | Train Loss: 0.6903, Train Acc: 0.5409 | Val Loss: 0.6890, Val Acc: 0.5473
Epoch 3/10 | Train Loss: 0.6903, Train Acc: 0.5408 | Val Loss: 0.6893, Val Acc: 0.5473
Epoch 4/10 | Train Loss: 0.6909, Train Acc: 0.5404 | Val Loss: 0.6889, Val Acc: 0.5473
Epoch 5/10 | Train Loss: 0.6899, Train Acc: 0.5411 | Val Loss: 0.6892, Val Acc: 0.5473
Epoch 6/10 | Train Loss: 0.6904, Train Acc: 0.5401 | Val Loss: 0.6892, Val Acc: 0.5473
Epoch 7/10 | Train Loss: 0.6902, Train Acc: 0.5409 | Val Loss: 0.6889, Val Acc: 0.5473
Epoch 8/10 | Train Loss: 0.6899, Train Acc: 0.5409 | Val Loss: 0.6893, Val Acc: 0.5473
Epoch 9/10 | Train Loss: 0.6904, Train Acc: 0.5408 | Val Loss: 0.6889, Val Acc: 0.5473
Epoch 10/10 | Train Loss: 0.6898, Train Acc: 0.5410 | Val Loss: 0.6889, Val Acc: 0.5473
Saved model for experiment 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 14 for fine-tuning.
Experiment 15: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6912, Train Acc: 0.5374 | Val Loss: 0.6913, Val Acc: 0.5307
Epoch 2/10 | Train Loss: 0.6906, Train Acc: 0.5358 | Val Loss: 0.6919, Val Acc: 0.5307
Epoch 3/10 | Train Loss: 0.6911, Train Acc: 0.5368 | Val Loss: 0.6913, Val Acc: 0.5307
Epoch 4/10 | Train Loss: 0.6909, Train Acc: 0.5374 | Val Loss: 0.6936, Val Acc: 0.5307
Epoch 5/10 | Train Loss: 0.6906, Train Acc: 0.5381 | Val Loss: 0.6917, Val Acc: 0.5307
Epoch 6/10 | Train Loss: 0.6907, Train Acc: 0.5380 | Val Loss: 0.6913, Val Acc: 0.5307
Epoch 7/10 | Train Loss: 0.6906, Train Acc: 0.5382 | Val Loss: 0.6917, Val Acc: 0.5307
Epoch 8/10 | Train Loss: 0.6903, Train Acc: 0.5384 | Val Loss: 0.6913, Val Acc: 0.5307
Epoch 9/10 | Train Loss: 0.6906, Train Acc: 0.5383 | Val Loss: 0.6913, Val Acc: 0.5307
Epoch 10/10 | Train Loss: 0.6906, Train Acc: 0.5383 | Val Loss: 0.6913, Val Acc: 0.5307
Saved model for experiment 

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt
import math

# 동일한 rolling_minmax_scale 및 bin_and_encode 함수 사용 (위와 동일)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

####################################
# Dataset 및 모델 정의는 Variant 1와 동일
####################################
class TimeSeriesDataset(Dataset):
    def __init__(self, input_data, target_data, lookback=24):
        self.input_data = input_data.values
        self.target_data = target_data.values
        self.lookback = lookback
    def __len__(self):
        return len(self.input_data) - self.lookback
    def __getitem__(self, idx):
        x = self.input_data[idx: idx + self.lookback, :]
        y = self.target_data[idx + self.lookback, 0]
        y_prev = self.target_data[idx + self.lookback - 1, 0]
        y_target = 1 if y > y_prev else 0
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_target, dtype=torch.long)

class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])

####################################
# Variant 2 실행: OHLC pct_change 기반
####################################
# CSV 파일 로드 – OHLC 데이터만 사용
data = pd.read_csv("BTC_upbit_KRW_min60.csv", index_col=0)
data = data[['open','high','low','close']]
data.index = pd.to_datetime(data.index)

# 계산: 각 feature에 대해 pct_change * 100
ohlc_features = ['open','high','low','close']
for feature in ohlc_features:
    data[feature + '_pct'] = data[feature].pct_change() * 100
# 클립: (-25, 25) 구간으로
for feature in ohlc_features:
    col = feature + '_pct'
    data[col] = data[col].clip(lower=-25, upper=25)
    
# one-hot 인코딩: pd.cut의 bins 인자는 np.linspace(-25,25,101)를 사용하여 100구간 생성
features_pct = [f + '_pct' for f in ohlc_features]
data = data.dropna()  # pct_change로 인해 생긴 결측 제거
data = bin_and_encode(data, features_pct, bins=100, drop_original=True)

# 타깃은 원본 close 값 사용 (원하는 경우 shift할 수도 있음)
data['close_target'] = data['close']
data = data.dropna()

# 최종 입력: _Bin_ 열들만 사용
final_input_columns = [col for col in data.columns if '_Bin_' in col]
final_target_column = ['close_target']

data_input = data[final_input_columns]
data_target = data[final_target_column]

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(data_loader), correct / total

def train_model(model, train_loader, val_loader, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def train_and_evaluate(data, num_experiments=16, lookback=24, num_epochs=10):
    # 최종 입력: 모든 _Bin_ 컬럼 + datetime one-hot + positional encoding + order 인코딩 컬럼
    final_input_columns = [col for col in data.columns if '_Bin_' in col]
    # datetime_onehot_features = [col for col in data.columns if col.startswith('Hour_') or 
    #                               col.startswith('Day_') or col.startswith('Week_') or 
    #                               col.startswith('Month_')]
    # datetime_positional_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos']
    # order_encoding_features = [col for col in data.columns if '_order_' in col]
    # final_input_columns.extend(datetime_onehot_features)
    # final_input_columns.extend(datetime_positional_features)
    # final_input_columns.extend(order_encoding_features)
    target_cols = ['close_target']
    
    data_input = data[final_input_columns]
    data_target = data[target_cols]
    
    data_input = data_input.apply(pd.to_numeric).astype(np.float32)
    data_target = data_target.apply(pd.to_numeric).astype(np.float32)
    
    step_size = 2500
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    val_acc_list = []
    test_acc_list = []
    
    for exp in range(num_experiments):
        train_start = exp * step_size
        train_end = train_start + step_size * 8
        val_end = train_end + step_size
        test_end = val_end + step_size
        if test_end > len(data_input):
            break
        print(f"Experiment {exp}: 데이터 구간 [{train_start}:{test_end}]")
        print(data)
        data.to_csv('AAPL_one_hot.csv')
        
        train_input = data_input.iloc[train_start:train_end]
        train_target = data_target.iloc[train_start:train_end]
        val_input = data_input.iloc[train_end:val_end]
        val_target = data_target.iloc[train_end:val_end]
        test_input = data_input.iloc[val_end:test_end]
        test_target = data_target.iloc[val_end:test_end]
        
        train_dataset = TimeSeriesDataset(train_input, train_target, lookback=lookback)
        val_dataset = TimeSeriesDataset(val_input, val_target, lookback=lookback)
        test_dataset = TimeSeriesDataset(test_input, test_target, lookback=lookback)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        lr = 1e-4
        input_dim = data_input.shape[1]
        model = EncoderOnlyTransformerCustom(input_dim=input_dim, embedding_dim=512, num_layers=6, nhead=8,
                                               ffn_dim=2048, num_classes=2, max_seq_len=lookback).to(device)
        model_path = f"model_experiment_{exp}.pth"
        if exp > 0:
            try:
                model.load_state_dict(torch.load(f"model_experiment_{exp - 1}.pth"))
                print(f"Loaded model from experiment {exp - 1} for fine-tuning.")
            except FileNotFoundError:
                print(f"Model file for experiment {exp - 1} not found. Starting fresh training.")
        
        print(f"Experiment {exp}: Training with lr={lr} (Fine-Tuning)")
        model = train_model(model, train_loader, val_loader, num_epochs, lr, device)
        torch.save(model.state_dict(), model_path)
        print(f"Saved model for experiment {exp}.")
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        val_acc_list.append(val_acc)
        print(f"Experiment {exp}: Final Validation Accuracy: {val_acc:.4f}")
        
        test_loss, test_acc = evaluate_model(model, test_loader, device)
        test_acc_list.append(test_acc)
        print(f"Experiment {exp}: Test Accuracy: {test_acc:.4f}")
    
    if val_acc_list:
        avg_val_acc = sum(val_acc_list) / len(val_acc_list)
        avg_test_acc = sum(test_acc_list) / len(test_acc_list)
        print(f"\nFinal Average Validation Accuracy: {avg_val_acc:.4f}")
        print(f"Final Average Test Accuracy: {avg_test_acc:.4f}")
    else:
        print("실험이 한 번도 실행되지 않았습니다.")

# 참조용 리스트 (최종 구성에는 _Bin_ 컬럼들이 포함됨)
# features_pct = [f + '_pct' for f in features_to_bin]
# features_minmax = [f + '_minmax' for f in features_to_bin]

train_and_evaluate(data)



C:\Users\User\AppData\Local\Temp\ipykernel_12720\1345690871.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['close_target'] = data['close']


Experiment 0: 데이터 구간 [0:25000]
                            open         high          low        close  \
2017-09-25 13:00:00    4222000.0    4235000.0    4208000.0    4235000.0   
2017-09-25 17:00:00    4235000.0    4235000.0    4235000.0    4235000.0   
2017-09-25 19:00:00    4213000.0    4213000.0    4187000.0    4204000.0   
2017-09-25 20:00:00    4204000.0    4204000.0    4175000.0    4191000.0   
2017-09-25 21:00:00    4191000.0    4218000.0    4183000.0    4211000.0   
...                          ...          ...          ...          ...   
2024-12-18 12:00:00  153867008.0  153870000.0  153000000.0  153452000.0   
2024-12-18 13:00:00  153451008.0  153452000.0  150348000.0  152200000.0   
2024-12-18 14:00:00  152200000.0  152800000.0  151382000.0  151994000.0   
2024-12-18 15:00:00  151904992.0  152500000.0  151524000.0  152184992.0   
2024-12-18 16:00:00  152186000.0  152696992.0  151999008.0  152559008.0   

                     open_pct  high_pct   low_pct  close_pct  open_p

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Experiment 0: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.7163, Train Acc: 0.4974 | Val Loss: 0.6942, Val Acc: 0.5053
Epoch 2/10 | Train Loss: 0.6978, Train Acc: 0.4957 | Val Loss: 0.6940, Val Acc: 0.4947
Epoch 3/10 | Train Loss: 0.6973, Train Acc: 0.5009 | Val Loss: 0.6933, Val Acc: 0.5053
Epoch 4/10 | Train Loss: 0.6983, Train Acc: 0.4948 | Val Loss: 0.6976, Val Acc: 0.4947
Epoch 5/10 | Train Loss: 0.6959, Train Acc: 0.5022 | Val Loss: 0.6936, Val Acc: 0.4947
Epoch 6/10 | Train Loss: 0.6956, Train Acc: 0.4996 | Val Loss: 0.6947, Val Acc: 0.4947
Epoch 7/10 | Train Loss: 0.6944, Train Acc: 0.5051 | Val Loss: 0.6940, Val Acc: 0.4947
Epoch 8/10 | Train Loss: 0.6949, Train Acc: 0.5021 | Val Loss: 0.6937, Val Acc: 0.4947
Epoch 9/10 | Train Loss: 0.6943, Train Acc: 0.4964 | Val Loss: 0.6950, Val Acc: 0.4947
Epoch 10/10 | Train Loss: 0.6938, Train Acc: 0.4996 | Val Loss: 0.6931, Val Acc: 0.5053
Saved model for experiment 0.
Experiment 0: Final Validation Accuracy: 0.5053

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


...

Loaded model from experiment 12 for fine-tuning.
Experiment 13: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6939, Train Acc: 0.5002 | Val Loss: 0.6940, Val Acc: 0.4818
Epoch 2/10 | Train Loss: 0.6941, Train Acc: 0.5003 | Val Loss: 0.6934, Val Acc: 0.4818
Epoch 3/10 | Train Loss: 0.6938, Train Acc: 0.4999 | Val Loss: 0.6928, Val Acc: 0.5182
Epoch 4/10 | Train Loss: 0.6934, Train Acc: 0.4985 | Val Loss: 0.6933, Val Acc: 0.4818
Epoch 5/10 | Train Loss: 0.6937, Train Acc: 0.5043 | Val Loss: 0.6939, Val Acc: 0.4818
Epoch 6/10 | Train Loss: 0.6937, Train Acc: 0.4995 | Val Loss: 0.6947, Val Acc: 0.4818
Epoch 7/10 | Train Loss: 0.6935, Train Acc: 0.5006 | Val Loss: 0.6934, Val Acc: 0.4818
Epoch 8/10 | Train Loss: 0.6936, Train Acc: 0.5007 | Val Loss: 0.6933, Val Acc: 0.4818
Epoch 9/10 | Train Loss: 0.6936, Train Acc: 0.4972 | Val Loss: 0.6942, Val Acc: 0.4818
Epoch 10/10 | Train Loss: 0.6932, Train Acc: 0.5018 | Val Loss: 0.6941, Val Acc: 0.4818
Saved model for experiment 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 13 for fine-tuning.
Experiment 14: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6939, Train Acc: 0.4946 | Val Loss: 0.6948, Val Acc: 0.4907
Epoch 2/10 | Train Loss: 0.6939, Train Acc: 0.4918 | Val Loss: 0.6941, Val Acc: 0.4907
Epoch 3/10 | Train Loss: 0.6938, Train Acc: 0.4978 | Val Loss: 0.6942, Val Acc: 0.4907
Epoch 4/10 | Train Loss: 0.6938, Train Acc: 0.5019 | Val Loss: 0.6931, Val Acc: 0.5093
Epoch 5/10 | Train Loss: 0.6935, Train Acc: 0.5005 | Val Loss: 0.6941, Val Acc: 0.4907
Epoch 6/10 | Train Loss: 0.6938, Train Acc: 0.5013 | Val Loss: 0.6937, Val Acc: 0.5093
Epoch 7/10 | Train Loss: 0.6934, Train Acc: 0.5057 | Val Loss: 0.6930, Val Acc: 0.5093
Epoch 8/10 | Train Loss: 0.6938, Train Acc: 0.5032 | Val Loss: 0.6951, Val Acc: 0.4907
Epoch 9/10 | Train Loss: 0.6938, Train Acc: 0.4986 | Val Loss: 0.6930, Val Acc: 0.5093
Epoch 10/10 | Train Loss: 0.6939, Train Acc: 0.4964 | Val Loss: 0.6937, Val Acc: 0.4907
Saved model for experiment 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 14 for fine-tuning.
Experiment 15: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6938, Train Acc: 0.5009 | Val Loss: 0.6932, Val Acc: 0.5000
Epoch 2/10 | Train Loss: 0.6936, Train Acc: 0.4987 | Val Loss: 0.6945, Val Acc: 0.5000
Epoch 3/10 | Train Loss: 0.6937, Train Acc: 0.4997 | Val Loss: 0.6931, Val Acc: 0.5000
Epoch 4/10 | Train Loss: 0.6937, Train Acc: 0.4992 | Val Loss: 0.6936, Val Acc: 0.5000
Epoch 5/10 | Train Loss: 0.6935, Train Acc: 0.5010 | Val Loss: 0.6933, Val Acc: 0.5000
Epoch 6/10 | Train Loss: 0.6934, Train Acc: 0.5052 | Val Loss: 0.6937, Val Acc: 0.5000
Epoch 7/10 | Train Loss: 0.6937, Train Acc: 0.4945 | Val Loss: 0.6933, Val Acc: 0.5000
Epoch 8/10 | Train Loss: 0.6933, Train Acc: 0.5017 | Val Loss: 0.6932, Val Acc: 0.5000
Epoch 9/10 | Train Loss: 0.6935, Train Acc: 0.4956 | Val Loss: 0.6932, Val Acc: 0.5000
Epoch 10/10 | Train Loss: 0.6933, Train Acc: 0.5033 | Val Loss: 0.6932, Val Acc: 0.5000
Saved model for experiment 

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt
import math

# 동일한 rolling_minmax_scale 및 bin_and_encode 함수 사용 (위와 동일)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

####################################
# Dataset 및 모델 정의는 Variant 1와 동일
####################################
class TimeSeriesDataset(Dataset):
    def __init__(self, input_data, target_data, lookback=24):
        self.input_data = input_data.values
        self.target_data = target_data.values
        self.lookback = lookback
    def __len__(self):
        return len(self.input_data) - self.lookback
    def __getitem__(self, idx):
        x = self.input_data[idx: idx + self.lookback, :]
        y = self.target_data[idx + self.lookback, 0]
        y_prev = self.target_data[idx + self.lookback - 1, 0]
        y_target = 1 if y > y_prev else 0
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_target, dtype=torch.long)

class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])

####################################
# Variant 2 실행: OHLC pct_change 기반
####################################
# CSV 파일 로드 – OHLC 데이터만 사용
data = pd.read_csv("BTC_upbit_KRW_min60.csv", index_col=0)
data = data[['open','high','low','close']]
data.index = pd.to_datetime(data.index)

# 계산: 각 feature에 대해 pct_change * 100
ohlc_features = ['open','high','low','close']
# 각 OHLC 열에 대해 rolling minmax scaling (window=24)
for feature in ohlc_features:
    data[feature] = rolling_minmax_scale(data[feature], window=24)

# One-hot 인코딩: 각 OHLC 열은 100구간으로 나눔
data = bin_and_encode(data, ohlc_features, bins=100, drop_original=True)

for feature in ohlc_features:
    data[feature + '_pct'] = data[feature].pct_change() * 100
# 클립: (-25, 25) 구간으로
for feature in ohlc_features:
    col = feature + '_pct'
    data[col] = data[col].clip(lower=-25, upper=25)
    
# one-hot 인코딩: pd.cut의 bins 인자는 np.linspace(-25,25,101)를 사용하여 100구간 생성
features_pct = [f + '_pct' for f in ohlc_features]
data = data.dropna()  # pct_change로 인해 생긴 결측 제거
data = bin_and_encode(data, features_pct, bins=100, drop_original=True)

# 타깃은 원본 close 값 사용 (원하는 경우 shift할 수도 있음)
data['close_target'] = data['close']
data = data.dropna()

# 최종 입력: _Bin_ 열들만 사용
final_input_columns = [col for col in data.columns if '_Bin_' in col]
final_target_column = ['close_target']

data_input = data[final_input_columns]
data_target = data[final_target_column]

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(data_loader), correct / total

def train_model(model, train_loader, val_loader, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def train_and_evaluate(data, num_experiments=16, lookback=24, num_epochs=10):
    # 최종 입력: 모든 _Bin_ 컬럼 + datetime one-hot + positional encoding + order 인코딩 컬럼
    final_input_columns = [col for col in data.columns if '_Bin_' in col]
    # datetime_onehot_features = [col for col in data.columns if col.startswith('Hour_') or 
    #                               col.startswith('Day_') or col.startswith('Week_') or 
    #                               col.startswith('Month_')]
    # datetime_positional_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos']
    # order_encoding_features = [col for col in data.columns if '_order_' in col]
    # final_input_columns.extend(datetime_onehot_features)
    # final_input_columns.extend(datetime_positional_features)
    # final_input_columns.extend(order_encoding_features)
    target_cols = ['close_target']
    
    data_input = data[final_input_columns]
    data_target = data[target_cols]
    
    data_input = data_input.apply(pd.to_numeric).astype(np.float32)
    data_target = data_target.apply(pd.to_numeric).astype(np.float32)
    
    step_size = 2500
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    val_acc_list = []
    test_acc_list = []
    
    for exp in range(num_experiments):
        train_start = exp * step_size
        train_end = train_start + step_size * 8
        val_end = train_end + step_size
        test_end = val_end + step_size
        if test_end > len(data_input):
            break
        print(f"Experiment {exp}: 데이터 구간 [{train_start}:{test_end}]")
        print(data)
        data.to_csv('AAPL_one_hot.csv')
        
        train_input = data_input.iloc[train_start:train_end]
        train_target = data_target.iloc[train_start:train_end]
        val_input = data_input.iloc[train_end:val_end]
        val_target = data_target.iloc[train_end:val_end]
        test_input = data_input.iloc[val_end:test_end]
        test_target = data_target.iloc[val_end:test_end]
        
        train_dataset = TimeSeriesDataset(train_input, train_target, lookback=lookback)
        val_dataset = TimeSeriesDataset(val_input, val_target, lookback=lookback)
        test_dataset = TimeSeriesDataset(test_input, test_target, lookback=lookback)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        lr = 1e-4
        input_dim = data_input.shape[1]
        model = EncoderOnlyTransformerCustom(input_dim=input_dim, embedding_dim=512, num_layers=6, nhead=8,
                                               ffn_dim=2048, num_classes=2, max_seq_len=lookback).to(device)
        model_path = f"model_experiment_{exp}.pth"
        if exp > 0:
            try:
                model.load_state_dict(torch.load(f"model_experiment_{exp - 1}.pth"))
                print(f"Loaded model from experiment {exp - 1} for fine-tuning.")
            except FileNotFoundError:
                print(f"Model file for experiment {exp - 1} not found. Starting fresh training.")
        
        print(f"Experiment {exp}: Training with lr={lr} (Fine-Tuning)")
        model = train_model(model, train_loader, val_loader, num_epochs, lr, device)
        torch.save(model.state_dict(), model_path)
        print(f"Saved model for experiment {exp}.")
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        val_acc_list.append(val_acc)
        print(f"Experiment {exp}: Final Validation Accuracy: {val_acc:.4f}")
        
        test_loss, test_acc = evaluate_model(model, test_loader, device)
        test_acc_list.append(test_acc)
        print(f"Experiment {exp}: Test Accuracy: {test_acc:.4f}")
    
    if val_acc_list:
        avg_val_acc = sum(val_acc_list) / len(val_acc_list)
        avg_test_acc = sum(test_acc_list) / len(test_acc_list)
        print(f"\nFinal Average Validation Accuracy: {avg_val_acc:.4f}")
        print(f"Final Average Test Accuracy: {avg_test_acc:.4f}")
    else:
        print("실험이 한 번도 실행되지 않았습니다.")

# 참조용 리스트 (최종 구성에는 _Bin_ 컬럼들이 포함됨)
# features_pct = [f + '_pct' for f in features_to_bin]
# features_minmax = [f + '_minmax' for f in features_to_bin]

train_and_evaluate(data)



C:\Users\User\AppData\Local\Temp\ipykernel_12720\777871736.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature + '_pct'] = data[feature].pct_change() * 100
C:\Users\User\AppData\Local\Temp\ipykernel_12720\777871736.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature + '_pct'] = data[feature].pct_change() * 100
C:\Users\User\AppData\Local\Temp\ipykernel_12720\777871736.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

Experiment 0: 데이터 구간 [0:25000]
                         open      high       low     close  open_Bin_0  \
2017-09-25 13:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 17:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 19:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 20:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
2017-09-25 21:00:00  1.000000  1.000000  1.000000  1.000000         0.0   
...                       ...       ...       ...       ...         ...   
2024-12-18 08:00:00  0.418090  0.421513  0.496974  0.467394         0.0   
2024-12-18 09:00:00  0.470346  0.519246  0.298104  0.226233         0.0   
2024-12-18 10:00:00  0.226700  0.344051  0.375151  0.006791         0.0   
2024-12-18 11:00:00  0.008636  0.000000  0.322711  0.000000         1.0   
2024-12-18 16:00:00  0.053565  0.041933  0.318664  0.109052         0.0   

                     open_Bin_1  open_Bin_2  open_Bin_3  open_Bin_4 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Experiment 0: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.7195, Train Acc: 0.5131 | Val Loss: 0.6981, Val Acc: 0.4580
Epoch 2/10 | Train Loss: 0.6956, Train Acc: 0.5186 | Val Loss: 0.7001, Val Acc: 0.4580
Epoch 3/10 | Train Loss: 0.6948, Train Acc: 0.5147 | Val Loss: 0.6901, Val Acc: 0.5420
Epoch 4/10 | Train Loss: 0.6887, Train Acc: 0.5208 | Val Loss: 0.6600, Val Acc: 0.5420
Epoch 5/10 | Train Loss: 0.6796, Train Acc: 0.5380 | Val Loss: 0.6722, Val Acc: 0.6026
Epoch 6/10 | Train Loss: 0.6914, Train Acc: 0.5318 | Val Loss: 0.6896, Val Acc: 0.5420
Epoch 7/10 | Train Loss: 0.6930, Train Acc: 0.5193 | Val Loss: 0.6897, Val Acc: 0.5420
Epoch 8/10 | Train Loss: 0.6922, Train Acc: 0.5212 | Val Loss: 0.6908, Val Acc: 0.5420
Epoch 9/10 | Train Loss: 0.6925, Train Acc: 0.5234 | Val Loss: 0.6895, Val Acc: 0.5420
Epoch 10/10 | Train Loss: 0.6916, Train Acc: 0.5249 | Val Loss: 0.6896, Val Acc: 0.5420
Saved model for experiment 0.
Experiment 0: Final Validation Accuracy: 0.5420

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


...

Loaded model from experiment 9 for fine-tuning.
Experiment 10: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6921, Train Acc: 0.5266 | Val Loss: 0.6956, Val Acc: 0.5234
Epoch 2/10 | Train Loss: 0.6923, Train Acc: 0.5259 | Val Loss: 0.6936, Val Acc: 0.5234
Epoch 3/10 | Train Loss: 0.6919, Train Acc: 0.5286 | Val Loss: 0.6924, Val Acc: 0.5234
Epoch 4/10 | Train Loss: 0.6926, Train Acc: 0.5286 | Val Loss: 0.6925, Val Acc: 0.5234
Epoch 5/10 | Train Loss: 0.6919, Train Acc: 0.5238 | Val Loss: 0.6954, Val Acc: 0.5234
Epoch 6/10 | Train Loss: 0.6920, Train Acc: 0.5288 | Val Loss: 0.6941, Val Acc: 0.4766
Epoch 7/10 | Train Loss: 0.6919, Train Acc: 0.5275 | Val Loss: 0.6925, Val Acc: 0.5234
Epoch 8/10 | Train Loss: 0.6919, Train Acc: 0.5298 | Val Loss: 0.6924, Val Acc: 0.5234
Epoch 9/10 | Train Loss: 0.6917, Train Acc: 0.5300 | Val Loss: 0.6922, Val Acc: 0.5234
Epoch 10/10 | Train Loss: 0.6914, Train Acc: 0.5287 | Val Loss: 0.6923, Val Acc: 0.5234
Saved model for experiment 1

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 10 for fine-tuning.
Experiment 11: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6925, Train Acc: 0.5216 | Val Loss: 0.6940, Val Acc: 0.5295
Epoch 2/10 | Train Loss: 0.6928, Train Acc: 0.5243 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 3/10 | Train Loss: 0.6923, Train Acc: 0.5243 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 4/10 | Train Loss: 0.6926, Train Acc: 0.5235 | Val Loss: 0.6915, Val Acc: 0.5295
Epoch 5/10 | Train Loss: 0.6922, Train Acc: 0.5239 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 6/10 | Train Loss: 0.6926, Train Acc: 0.5209 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 7/10 | Train Loss: 0.6921, Train Acc: 0.5225 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 8/10 | Train Loss: 0.6925, Train Acc: 0.5210 | Val Loss: 0.6918, Val Acc: 0.5295
Epoch 9/10 | Train Loss: 0.6924, Train Acc: 0.5259 | Val Loss: 0.6914, Val Acc: 0.5295
Epoch 10/10 | Train Loss: 0.6921, Train Acc: 0.5237 | Val Loss: 0.6919, Val Acc: 0.5295
Saved model for experiment 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Loaded model from experiment 11 for fine-tuning.
Experiment 12: Training with lr=0.0001 (Fine-Tuning)
Epoch 1/10 | Train Loss: 0.6923, Train Acc: 0.5195 | Val Loss: 0.6935, Val Acc: 0.5226
Epoch 2/10 | Train Loss: 0.6923, Train Acc: 0.5260 | Val Loss: 0.6933, Val Acc: 0.5226
Epoch 3/10 | Train Loss: 0.6923, Train Acc: 0.5227 | Val Loss: 0.6929, Val Acc: 0.5226
Epoch 4/10 | Train Loss: 0.6923, Train Acc: 0.5219 | Val Loss: 0.6922, Val Acc: 0.5226
Epoch 5/10 | Train Loss: 0.6924, Train Acc: 0.5244 | Val Loss: 0.6948, Val Acc: 0.5226
Epoch 6/10 | Train Loss: 0.6923, Train Acc: 0.5231 | Val Loss: 0.6927, Val Acc: 0.5226
Epoch 7/10 | Train Loss: 0.6921, Train Acc: 0.5244 | Val Loss: 0.6930, Val Acc: 0.5226
Epoch 8/10 | Train Loss: 0.6918, Train Acc: 0.5279 | Val Loss: 0.6922, Val Acc: 0.5226
Epoch 9/10 | Train Loss: 0.6919, Train Acc: 0.5265 | Val Loss: 0.6924, Val Acc: 0.5226
Epoch 10/10 | Train Loss: 0.6921, Train Acc: 0.5261 | Val Loss: 0.6923, Val Acc: 0.5226
Saved model for experiment 

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt

# rolling minmax scaling 함수 (window=24)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

# binning 및 one-hot 인코딩 함수 (결과를 정수 0,1로)
def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        # pd.cut에 bins로 지정된 구간을 사용 (여기서는 자동으로 최소~최대 값)
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    # 개별 열 단위로 float32로 변환
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

####################################
# Dataset 및 모델 정의 (나머지 구성은 동일)
####################################
class TimeSeriesDataset(Dataset):
    def __init__(self, input_data, target_data, lookback=24):
        self.input_data = input_data.values
        self.target_data = target_data.values
        self.lookback = lookback

    def __len__(self):
        return len(self.input_data) - self.lookback

    def __getitem__(self, idx):
        x = self.input_data[idx: idx + self.lookback, :]
        y = self.target_data[idx + self.lookback, 0]
        y_prev = self.target_data[idx + self.lookback - 1, 0]
        y_target = 1 if y > y_prev else 0
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_target, dtype=torch.long)

class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])

####################################
# 데이터 로드 및 Variant 1 실행: OHLC 원본 데이터로 실험
####################################
# CSV 파일 로드 – OHLC 4개 데이터만 사용
data = pd.read_csv("ETH_upbit_KRW_min5.csv", index_col=0)
data = data[['open', 'high', 'low', 'close']]
data.index = pd.to_datetime(data.index)

# 각 OHLC 열에 대해 rolling minmax scaling (window=24)
ohlc_features = ['open', 'high', 'low', 'close']
for feature in ohlc_features:
    data[feature] = rolling_minmax_scale(data[feature], window=24)

# One-hot 인코딩: 각 OHLC 열은 100구간으로 나눔
data = bin_and_encode(data, ohlc_features, bins=100, drop_original=True)

# 타깃은 원본 close 값 사용 (실험 목적)
data['close_target'] = data['close']
data = data.dropna()

# 최종 입력은 _Bin_ 접미사 열들만 사용
final_input_columns = [col for col in data.columns if '_Bin_' in col]
final_target_column = ['close_target']

data_input = data[final_input_columns]
data_target = data[final_target_column]


def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(data_loader), correct / total

def train_model(model, train_loader, val_loader, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def train_and_evaluate(data, num_experiments=16, lookback=24, num_epochs=10):
    final_input_columns = [col for col in data.columns if '_Bin_' in col]
    target_cols = ['close_target']
    
    data_input = data[final_input_columns]
    data_target = data[target_cols]
    
    data_input = data_input.apply(pd.to_numeric).astype(np.float32)
    data_target = data_target.apply(pd.to_numeric).astype(np.float32)
    
    step_size = 25000
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    val_acc_list = []
    test_acc_list = []
    
    for exp in range(num_experiments):
        train_start = exp * step_size
        train_end = train_start + step_size * 8
        val_end = train_end + step_size
        test_end = val_end + step_size
        if test_end > len(data_input):
            break
        print(f"Experiment {exp}: 데이터 구간 [{train_start}:{test_end}]")
        print(data)
        
        train_input = data_input.iloc[train_start:train_end]
        train_target = data_target.iloc[train_start:train_end]
        val_input = data_input.iloc[train_end:val_end]
        val_target = data_target.iloc[train_end:val_end]
        test_input = data_input.iloc[val_end:test_end]
        test_target = data_target.iloc[val_end:test_end]
        
        train_dataset = TimeSeriesDataset(train_input, train_target, lookback=lookback)
        val_dataset = TimeSeriesDataset(val_input, val_target, lookback=lookback)
        test_dataset = TimeSeriesDataset(test_input, test_target, lookback=lookback)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        lr = 1e-4
        input_dim = data_input.shape[1]
        model = EncoderOnlyTransformerCustom(input_dim=input_dim, embedding_dim=512, num_layers=6, nhead=8,
                                               ffn_dim=2048, num_classes=2, max_seq_len=lookback).to(device)
        model_path = f"model_experiment_{exp}.pth"
        if exp > 0:
            try:
                model.load_state_dict(torch.load(f"model_experiment_{exp - 1}.pth"))
                print(f"Loaded model from experiment {exp - 1} for fine-tuning.")
            except FileNotFoundError:
                print(f"Model file for experiment {exp - 1} not found. Starting fresh training.")
        
        print(f"Experiment {exp}: Training with lr={lr} (Fine-Tuning)")
        model = train_model(model, train_loader, val_loader, num_epochs, lr, device)
        torch.save(model.state_dict(), model_path)
        print(f"Saved model for experiment {exp}.")
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        val_acc_list.append(val_acc)
        print(f"Experiment {exp}: Final Validation Accuracy: {val_acc:.4f}")
        
        test_loss, test_acc = evaluate_model(model, test_loader, device)
        test_acc_list.append(test_acc)
        print(f"Experiment {exp}: Test Accuracy: {test_acc:.4f}")
    
    if val_acc_list:
        avg_val_acc = sum(val_acc_list) / len(val_acc_list)
        avg_test_acc = sum(test_acc_list) / len(test_acc_list)
        print(f"\nFinal Average Validation Accuracy: {avg_val_acc:.4f}")
        print(f"Final Average Test Accuracy: {avg_test_acc:.4f}")
    else:
        print("실험이 한 번도 실행되지 않았습니다.")


train_and_evaluate(data)


C:\Users\User\AppData\Local\Temp\ipykernel_26012\2793618170.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['close_target'] = data['close']


Experiment 0: 데이터 구간 [0:250000]
                         open      high       low  close  open_Bin_0  \
2017-09-25 12:00:00  1.000000  1.000000  1.000000   1.00         0.0   
2017-09-25 12:05:00  1.000000  1.000000  1.000000   1.00         0.0   
2017-09-25 12:10:00  1.000000  1.000000  1.000000   1.00         0.0   
2017-09-25 12:15:00  1.000000  1.000000  1.000000   1.00         0.0   
2017-09-25 12:20:00  1.000000  1.000000  1.000000   1.00         0.0   
...                       ...       ...       ...    ...         ...   
2024-12-18 16:05:00  0.200000  0.500000  0.462963   0.54         0.0   
2024-12-18 16:10:00  0.490909  0.685185  0.648148   0.50         0.0   
2024-12-18 16:15:00  0.472727  0.518519  0.759259   0.58         0.0   
2024-12-18 16:20:00  0.527273  0.648148  0.814815   0.64         0.0   
2024-12-18 16:25:00  0.581818  0.611111  0.870370   0.70         0.0   

                     open_Bin_1  open_Bin_2  open_Bin_3  open_Bin_4  \
2017-09-25 12:00:00         0.0 

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_produc

Epoch 1/10 | Train Loss: 0.6845, Train Acc: 0.5763 | Val Loss: 0.6772, Val Acc: 0.5897
Epoch 2/10 | Train Loss: 0.6811, Train Acc: 0.5810 | Val Loss: 0.6777, Val Acc: 0.5897
Epoch 3/10 | Train Loss: 0.6807, Train Acc: 0.5810 | Val Loss: 0.6771, Val Acc: 0.5897
Epoch 4/10 | Train Loss: 0.6809, Train Acc: 0.5810 | Val Loss: 0.6816, Val Acc: 0.5897
Epoch 5/10 | Train Loss: 0.6809, Train Acc: 0.5811 | Val Loss: 0.6773, Val Acc: 0.5897
Epoch 6/10 | Train Loss: 0.6808, Train Acc: 0.5811 | Val Loss: 0.6770, Val Acc: 0.5897
Epoch 7/10 | Train Loss: 0.6807, Train Acc: 0.5810 | Val Loss: 0.6771, Val Acc: 0.5897
Epoch 8/10 | Train Loss: 0.6808, Train Acc: 0.5811 | Val Loss: 0.6771, Val Acc: 0.5897
Epoch 9/10 | Train Loss: 0.6807, Train Acc: 0.5811 | Val Loss: 0.6770, Val Acc: 0.5897
Epoch 10/10 | Train Loss: 0.6805, Train Acc: 0.5811 | Val Loss: 0.6773, Val Acc: 0.5897
Saved model for experiment 0.
Experiment 0: Final Validation Accuracy: 0.5897
Experiment 0: Test Accuracy: 0.5949
Experiment 1: 데

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


...

Epoch 1/10 | Train Loss: 0.6818, Train Acc: 0.5754 | Val Loss: 0.6735, Val Acc: 0.6021
Epoch 2/10 | Train Loss: 0.6818, Train Acc: 0.5754 | Val Loss: 0.6727, Val Acc: 0.6021
Epoch 3/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6728, Val Acc: 0.6021
Epoch 4/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6739, Val Acc: 0.6021
Epoch 5/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6726, Val Acc: 0.6021
Epoch 6/10 | Train Loss: 0.6818, Train Acc: 0.5754 | Val Loss: 0.6733, Val Acc: 0.6021
Epoch 7/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6732, Val Acc: 0.6021
Epoch 8/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6729, Val Acc: 0.6021
Epoch 9/10 | Train Loss: 0.6819, Train Acc: 0.5754 | Val Loss: 0.6741, Val Acc: 0.6021
Epoch 10/10 | Train Loss: 0.6818, Train Acc: 0.5754 | Val Loss: 0.6732, Val Acc: 0.6021
Saved model for experiment 13.
Experiment 13: Final Validation Accuracy: 0.6021
Experiment 13: Test Accuracy: 0.6071
Experiment 1

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6732, Val Acc: 0.6071
Epoch 2/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6715, Val Acc: 0.6071
Epoch 3/10 | Train Loss: 0.6805, Train Acc: 0.5801 | Val Loss: 0.6705, Val Acc: 0.6071
Epoch 4/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6713, Val Acc: 0.6071
Epoch 5/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6713, Val Acc: 0.6071
Epoch 6/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6710, Val Acc: 0.6071
Epoch 7/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6714, Val Acc: 0.6071
Epoch 8/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6710, Val Acc: 0.6071
Epoch 9/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6711, Val Acc: 0.6071
Epoch 10/10 | Train Loss: 0.6804, Train Acc: 0.5801 | Val Loss: 0.6713, Val Acc: 0.6071
Saved model for experiment 14.
Experiment 14: Final Validation Accuracy: 0.6071
Experiment 14: Test Accuracy: 0.6274
Experiment 1

c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10 | Train Loss: 0.6793, Train Acc: 0.5835 | Val Loss: 0.6639, Val Acc: 0.6274
Epoch 2/10 | Train Loss: 0.6793, Train Acc: 0.5835 | Val Loss: 0.6644, Val Acc: 0.6274
Epoch 3/10 | Train Loss: 0.6793, Train Acc: 0.5835 | Val Loss: 0.6648, Val Acc: 0.6274
Epoch 4/10 | Train Loss: 0.6793, Train Acc: 0.5835 | Val Loss: 0.6648, Val Acc: 0.6274
Epoch 5/10 | Train Loss: 0.6793, Train Acc: 0.5835 | Val Loss: 0.6639, Val Acc: 0.6274
Epoch 6/10 | Train Loss: 0.6792, Train Acc: 0.5835 | Val Loss: 0.6635, Val Acc: 0.6274
Epoch 7/10 | Train Loss: 0.6792, Train Acc: 0.5835 | Val Loss: 0.6630, Val Acc: 0.6274
Epoch 8/10 | Train Loss: 0.6792, Train Acc: 0.5835 | Val Loss: 0.6637, Val Acc: 0.6274
Epoch 9/10 | Train Loss: 0.6792, Train Acc: 0.5835 | Val Loss: 0.6629, Val Acc: 0.6274
Epoch 10/10 | Train Loss: 0.6792, Train Acc: 0.5835 | Val Loss: 0.6633, Val Acc: 0.6274
Saved model for experiment 15.
Experiment 15: Final Validation Accuracy: 0.6274
Experiment 15: Test Accuracy: 0.6484

Final Avera

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt

# rolling minmax scaling 함수 (window=24)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

# binning 및 one-hot 인코딩 함수 (결과를 정수 0,1로)
def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        # pd.cut에 bins로 지정된 구간을 사용 (여기서는 자동으로 최소~최대 값)
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    # 개별 열 단위로 float32로 변환
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

####################################
# Dataset 및 모델 정의 (나머지 구성은 동일)
####################################
class TimeSeriesDataset(Dataset):
    def __init__(self, input_data, target_data, lookback=24):
        self.input_data = input_data.values
        self.target_data = target_data.values
        self.lookback = lookback

    def __len__(self):
        return len(self.input_data) - self.lookback

    def __getitem__(self, idx):
        x = self.input_data[idx: idx + self.lookback, :]
        y = self.target_data[idx + self.lookback, 0]
        y_prev = self.target_data[idx + self.lookback - 1, 0]
        y_target = 1 if y > y_prev else 0
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_target, dtype=torch.long)

class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])

####################################
# 데이터 로드 및 Variant 1 실행: OHLC 원본 데이터로 실험
####################################
# CSV 파일 로드 – OHLC 4개 데이터만 사용
data = pd.read_csv("ETH_upbit_KRW_min5_0309.csv", index_col=0)
data = data[['open', 'high', 'low', 'close']]
data.index = pd.to_datetime(data.index)

# 각 OHLC 열에 대해 rolling minmax scaling (window=24)
ohlc_features = ['open', 'high', 'low', 'close']
for feature in ohlc_features:
    data[feature] = rolling_minmax_scale(data[feature], window=24)

# One-hot 인코딩: 각 OHLC 열은 100구간으로 나눔
data = bin_and_encode(data, ohlc_features, bins=100, drop_original=True)

# 타깃은 원본 close 값 사용 (실험 목적)
data['close_target'] = data['close']
data = data.dropna()

# 최종 입력은 _Bin_ 접미사 열들만 사용
final_input_columns = [col for col in data.columns if '_Bin_' in col]
final_target_column = ['close_target']

data_input = data[final_input_columns]
data_target = data[final_target_column]


def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(data_loader), correct / total

def train_model(model, train_loader, val_loader, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def train_and_evaluate(data, num_experiments=16, lookback=24, num_epochs=10):
    final_input_columns = [col for col in data.columns if '_Bin_' in col]
    target_cols = ['close_target']
    
    data_input = data[final_input_columns]
    data_target = data[target_cols]
    
    data_input = data_input.apply(pd.to_numeric).astype(np.float32)
    data_target = data_target.apply(pd.to_numeric).astype(np.float32)
    
    step_size = 31200
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    val_acc_list = []
    test_acc_list = []
    
    for exp in range(num_experiments):
        train_start = exp * step_size
        train_end = train_start + step_size * 8
        val_end = train_end + step_size
        test_end = val_end + step_size
        if test_end > len(data_input):
            break
        print(f"Experiment {exp}: 데이터 구간 [{train_start}:{test_end}]")
        print(data)
        
        train_input = data_input.iloc[train_start:train_end]
        train_target = data_target.iloc[train_start:train_end]
        val_input = data_input.iloc[train_end:val_end]
        val_target = data_target.iloc[train_end:val_end]
        test_input = data_input.iloc[val_end:test_end]
        test_target = data_target.iloc[val_end:test_end]
        
        train_dataset = TimeSeriesDataset(train_input, train_target, lookback=lookback)
        val_dataset = TimeSeriesDataset(val_input, val_target, lookback=lookback)
        test_dataset = TimeSeriesDataset(test_input, test_target, lookback=lookback)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        lr = 1e-4
        input_dim = data_input.shape[1]
        model = EncoderOnlyTransformerCustom(input_dim=input_dim, embedding_dim=512, num_layers=6, nhead=8,
                                               ffn_dim=2048, num_classes=2, max_seq_len=lookback).to(device)
        model_path = f"model_experiment_{exp}.pth"
        if exp > 0:
            try:
                model.load_state_dict(torch.load(f"model_experiment_{exp - 1}.pth"))
                print(f"Loaded model from experiment {exp - 1} for fine-tuning.")
            except FileNotFoundError:
                print(f"Model file for experiment {exp - 1} not found. Starting fresh training.")
        
        print(f"Experiment {exp}: Training with lr={lr} (Fine-Tuning)")
        model = train_model(model, train_loader, val_loader, num_epochs, lr, device)
        torch.save(model.state_dict(), model_path)
        print(f"Saved model for experiment {exp}.")
        
        val_loss, val_acc = evaluate_model(model, val_loader, device)
        val_acc_list.append(val_acc)
        print(f"Experiment {exp}: Final Validation Accuracy: {val_acc:.4f}")
        
        test_loss, test_acc = evaluate_model(model, test_loader, device)
        test_acc_list.append(test_acc)
        print(f"Experiment {exp}: Test Accuracy: {test_acc:.4f}")
    
    if val_acc_list:
        avg_val_acc = sum(val_acc_list) / len(val_acc_list)
        avg_test_acc = sum(test_acc_list) / len(test_acc_list)
        print(f"\nFinal Average Validation Accuracy: {avg_val_acc:.4f}")
        print(f"Final Average Test Accuracy: {avg_test_acc:.4f}")
    else:
        print("실험이 한 번도 실행되지 않았습니다.")


train_and_evaluate(data)
